<h1>Import Python Packages</h1>

In [1]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/metatlas/metatlas/helpers/' )
import dill2plots as dp
%matplotlib inline

('Metatlas live in ', '/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/metatlas')
you're running on 072ec0638adb at 172.17.0.4 


/anaconda2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Make blank file-info sheet

Get files from database

In [2]:
#Experiment is your folder name
#name is your file name
files = dp.get_metatlas_files(experiment = '%angelo%',name = '%PA%',most_recent = True)

In [3]:
len(files)

344

Save the files to an empty fileinfo sheet

In [ ]:
dp.make_empty_fileinfo_sheet('/global/homes/b/bpb/Downloads/empty_ophelia_finfo.tab',files)

# Create metatlas groups from filled in file-info sheet

In [ ]:
dp = reload(dp)
g = dp.make_groups_from_fileinfo_sheet('/global/homes/b/bpb/Downloads/20160517_RL_HM_6550_HILIC_CA-MF-groups.csv',
                                       filetype='csv',
                                       store=False)

# Create a new Atlas in metatlas from a spreadsheet

In [ ]:
names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/Not_Found_BrianHedlund_Atlas_POS_using_MSMS.csv',
                                       'hedlund_hilic_qexactive_pos_not observed',
                                       filetype='csv',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=False,
#                                        dataframe = df3,
                                      mz_tolerance = 20)

In [ ]:
# import pandas as pd
# path = '/global/homes/b/bpb/Downloads/hedlund/'
# input_file = 'BrianHedlund_Atlas_POS_using_MSMS.csv'
# df = pd.read_csv(path + input_file)
# df.loc[df.name == 'Arsenobetaine','name'] = ''
# output_file = 'Not_Found_' + input_file
# print path+output_file
# df[df.new_observed == 0][['name','label','rt_min','rt_max','rt_peak','mz','mz_tolerance']].to_csv(path+output_file)

# output_file = 'RT_and_MSMS_' + input_file
# print path+output_file
# df[df.new_observed == 1][['name','label','rt_min','rt_max','rt_peak','mz','mz_tolerance']].to_csv(path+output_file)

# output_file = 'RT_' + input_file
# print path+output_file
# df[df.new_observed == 2][['name','label','rt_min','rt_max','rt_peak','mz','mz_tolerance']].to_csv(path+output_file)

In [ ]:
# dp = reload(dp)
# # df3 = dp.get_formatted_atlas_from_google_sheet(polarity='POS',method='QE_HILIC', mz_tolerance=10)
# names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/Not_Found_BrianHedlund_Atlas_POS_using_MSMS.csv',
#                                        'hedlund_hilic_qexactive_pos_not observed',
#                                        filetype='csv',
#                                        sheetname='',
#                                        polarity = 'positive',
#                                        store=True,
# #                                        dataframe = df3,
#                                       mz_tolerance = 20)

# names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/RT_and_MSMS_BrianHedlund_Atlas_POS_using_MSMS.csv',
#                                        'hedlund_hilic_qexactive_pos_present',
#                                        filetype='csv',
#                                        sheetname='',
#                                        polarity = 'positive',
#                                        store=True,
# #                                        dataframe = df3,
#                                       mz_tolerance = 20)

# names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/RT_BrianHedlund_Atlas_POS_using_MSMS.csv',
#                                        'hedlund_hilic_qexactive_pos_maybe',
#                                        filetype='csv',
#                                        sheetname='',
#                                        polarity = 'positive',
#                                        store=True,
# #                                        dataframe = df3,
#                                       mz_tolerance = 20)

# names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/Not_Found_BrianHedlund_Atlas_NEG_using_MSMS.csv',
#                                        'hedlund_hilic_qexactive_neg_not observed',
#                                        filetype='csv',
#                                        sheetname='',
#                                        polarity = 'negative',
#                                        store=True,
# #                                        dataframe = df3,
#                                       mz_tolerance = 20)

# names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/RT_and_MSMS_BrianHedlund_Atlas_NEG_using_MSMS.csv',
#                                        'hedlund_hilic_qexactive_neg_present',
#                                        filetype='csv',
#                                        sheetname='',
#                                        polarity = 'negative',
#                                        store=True,
# #                                        dataframe = df3,
#                                       mz_tolerance = 20)

# names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/RT_BrianHedlund_Atlas_NEG_using_MSMS.csv',
#                                        'hedlund_hilic_qexactive_neg_maybe',
#                                        filetype='csv',
#                                        sheetname='',
#                                        polarity = 'negative',
#                                        store=True,
# #                                        dataframe = df3,
#                                       mz_tolerance = 20)

In [1]:
# atlases = ['hedlund_hilic_qexactive_neg_not observed',
#         'hedlund_hilic_qexactive_neg_present',
#         'hedlund_hilic_qexactive_neg_maybe',
#         'hedlund_hilic_qexactive_pos_not observed',
#         'hedlund_hilic_qexactive_pos_maybe',
#         'hedlund_hilic_qexactive_pos_present']


# import os           

# for a in atlases:
# #     if 'neg' in a:
# #         group_str = '%hedlund%neg%'
# #     else:
# #         group_str = '%hedlund%pos%'
# #     groups = dp.select_groups_for_analysis(name = group_str, most_recent = True, remove_empty = True, filter_list = ['Set1'], exclude_list = ['QC','Blank'])
        
# #     atlas = dp.get_metatlas_atlas(name=a,do_print = True)
    
# #     TODO: Export the searched data instead of the atlas
# #     df = dp.export_atlas_to_spreadsheet(atlas[0],'/global/homes/b/bpb/Downloads/hedlund/set1/%s.csv'%a)
#     output_filename = '/global/homes/b/bpb/Downloads/hedlund/set1/pickle_%s.pkl'%a
#     print output_filename
# #     data = dp.get_data_for_groups_and_atlas(groups,atlas[0],output_filename,use_set1 = True)
    
#     my_str = os.path.basename(output_filename).split('.')[0].replace(' ','_').replace('pickle_hedlund_','')
#     project_label = '/global/homes/b/bpb/Downloads/hedlund/set1/%s'%my_str
#     print project_label
#     peak_height = dp.make_output_dataframe(input_fname = output_filename, fieldname='peak_height' , output_loc=project_label+'/sheets')
# #     peak_area = dp.make_output_dataframe(input_fname = output_filename, fieldname='peak_area' , output_loc=project_label+'/sheets')
# #     rt_peak = dp.make_output_dataframe(input_fname = output_filename, fieldname='rt_peak' , output_loc=project_label+'/sheets')
# #     mz_centroid = dp.make_output_dataframe(input_fname = output_filename, fieldname='mz_centroid' , output_loc=project_label+'/sheets')

#     dp.plot_errorbar_plots(peak_height,input_fname = output_filename, output_loc=project_label+'/error_bar_peak_height')
# #     dp.plot_all_files_for_each_compound(input_fname = output_filename, nCols = 6, scale_y=False , output_loc=project_label+'/all_files_each_compound')
# #     dp.make_identification_figure(input_fname = output_filename, output_loc=project_label+'/identification')

#Select groups of files to operate on

In [ ]:
dp = reload(dp)

groups = dp.select_groups_for_analysis(name = '%RL_HM_6550_HILIC_20160517_%Neg',
                                       most_recent = True,
                                       remove_empty = True,
                                       filter_list = [])#, exclude_list = ['QC','Blank'])
# for g in groups:
#     print g.name

#Select Atlas to use

In [5]:
dp = reload(dp)
atlas = dp.get_metatlas_atlas(name='%RL_HM_%neg%',do_print = True)

0 50 RL_HM_6550_20160330_Neg_Hilic_Arkin_Ophelia 2016-04-05 14:34:53


In [ ]:
myAtlas = atlas[0]
myAtlas.name

#Export Atlas To Spreadsheet

In [ ]:
dp = reload(dp)
df = dp.export_atlas_to_spreadsheet(myAtlas,'/global/homes/b/bpb/Downloads/ophelia_20160517/neg_atlas.csv',
                                    compound_lookup='/project/projectdirs/metatlas/uniquecompounds.csv')
df.head()

#Get Data and Save it

In [ ]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
dp = reload(dp)
output_filename = '/global/homes/b/bpb/Downloads/ophelia_20160517/neg_data.pkl'
data = dp.get_data_for_groups_and_atlas(groups,myAtlas,output_filename,use_set1 = False)

In [ ]:
# my_files = ['/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_POS_Present.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_POS_Maybe.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_POS_Not_Observed.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_NEG_Present.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_NEG_Maybe.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_NEG_Not_Observed.pkl']
my_files = ['/global/homes/b/bpb/Downloads/ophelia_20160517/neg_data.pkl',
            '/global/homes/b/bpb/Downloads/ophelia_20160517/pos_data.pkl']
import os           
dp = reload(dp)
for my_file in my_files:
    project_label = '/global/homes/b/bpb/Downloads/ophelia_20160517/%s'%os.path.basename(my_file).split('.')[0]
    peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')
    dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')
    dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 6, scale_y=True , output_loc=project_label+'/scaled_all_compounds_each_file')
    dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=True , output_loc=project_label+'/scaled_all_files_each_compound')
    dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 6, scale_y=False , output_loc=project_label+'/all_compounds_each_file')
    dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=False , output_loc=project_label+'/all_files_each_compound')
    dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')

#Load a pre-existing datafile

In [ ]:
output_filename = '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_POS_Present.pkl'
my_file = output_filename #'/global/homes/b/bpb/notebooks/MetAtlas_Workflow_20151123/20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'

project_label = '/global/homes/b/bpb/Downloads/hedlund/TestIDFigs'

#Make Supplementary Tables

In [ ]:
peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')

#Error bar

In [ ]:
dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')

#Make Chromatograms

In [ ]:
dp = reload(dp)
dp.plot_all_compounds_for_each_file(input_fname = my_file, 
                                    nCols = 8, 
                                    scale_y=True , 
                                    output_loc=project_label+'/scaled_all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=True , output_loc=project_label+'/scaled_all_files_each_compound')
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 8, scale_y=False , output_loc=project_label+'/all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=False , output_loc=project_label+'/all_files_each_compound')

#Make Identification Figures

In [ ]:
dp = reload(dp)
dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')